In [1]:
OUT_DIR = '/tmp/'

from torchvision import models
import torchvision.datasets as dsets
import json
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from robustness import model_utils, datasets, train, defaults
from robustness.datasets import CIFAR, ImageNet
import torch
from cox.utils import Parameters
import cox.store
from torchvision import models
import torch
from torchvision import transforms
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
# Standard libraries
import numpy as np
import os
# PyTorch
import torch.nn as nn
from PIL import Image

/home/hashmat/miniconda3/envs/advdrop_env/lib/python3.7/site-packages/robustness/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


In [2]:

class_idx = json.load(open("/home/hashmat/Thesis/AdvDrop/imagenet_class_index.json"))
idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
class2label = [class_idx[str(k)][0] for k in range(len(class_idx))]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), ])


def image_folder_custom_label(root, transform, idx2label) :
    
    
    old_data = dsets.ImageFolder(root=root, transform=transform)
    old_classes = old_data.classes
    
    label2idx = {}
    
    for i, item in enumerate(idx2label) :
        label2idx[item] = i
    
    new_data = dsets.ImageFolder(root=root, transform=transform, 
                                 target_transform=lambda x : idx2label.index(old_classes[x]))
    new_data.classes = idx2label
    new_data.class_to_idx = label2idx

    return new_data

def create_dir(dir, print_flag = False):
    if not os.path.exists(dir):
        os.mkdir(dir)
        if print_flag:
            print("Create dir {} successfully!".format(dir))
    elif print_flag:
        print("Directory {} is already existed. ".format(dir))


def get_data(data_dir, transform, batch_size, class2label):

    normal_data = image_folder_custom_label(root=data_dir,
                                            transform=transform,
                                            idx2label=class2label)
    normal_loader = torch.utils.data.DataLoader(normal_data,
                                                batch_size=batch_size,
                                                shuffle=False)

    return normal_loader

In [3]:
#from Models.transformers import diet_tiny, diet_small, vit_tiny, vit_small
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.register_buffer('mean', torch.Tensor(mean))
        self.register_buffer('std', torch.Tensor(std))

    def forward(self, input):
        # Broadcasting
        input = input / 255.0
        mean = self.mean.reshape(1, 3, 1, 1)
        std = self.std.reshape(1, 3, 1, 1)
        return (input - mean) / std

transform = transforms.Compose([
transforms.Resize((224, 224)),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406],
                   std=[0.229, 0.224, 0.225])])


def evaluate_accuracy(model, bs=128, 
                 data_path="/home/hashmat/Thesis/nobox-attacks/data/ILSVRC2012_img_val",
                     tf = transform):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    OUT_DIR = '/tmp/'
    imagenet_ds = ImageNet('/tmp/')
    model = model.eval()
    batch_size = bs
    data_dir = data_path
    normal_loader = get_data(data_dir, tf, batch_size, class2label)
    correct = 0
    total = 0
    count = 0
    for images, labels in normal_loader:
        images, labels = images.to(device), labels.to(device)
        preds = model(images, with_image=False)
        _, pre = torch.max(preds, 1)
        correct += torch.sum(pre==labels)
        total += images.shape[0]
        print(total)
    print(f"The Model accuracy is {correct/total}")

   


## Load VGG Models

In [ ]:
imagenet_ds = ImageNet('/tmp/')
vgg , _ = model_utils.make_and_restore_model(arch=models.vgg16_bn(), dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/vgg16_bn_l2_eps0.ckpt',
                                parallel=False, add_custom_forward=True)
model = vgg.model
evauate_accuracy(model, bs=4)

## Load MobileNet-v2

In [ ]:
# L-2 epsilon = 0 

In [ ]:
imagenet_ds = ImageNet('/tmp/')
mobilenet , _ = model_utils.make_and_restore_model(arch=models.mobilenet_v2(), dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/mobilenet_l2_eps0.ckpt',
                                parallel=False, add_custom_forward=True)
model = mobilenet.model
evaluate_accuracy(model, bs=4)

In [ ]:
# L-2 epsilon = 3

In [ ]:
imagenet_ds = ImageNet('/tmp/')
mobilenet , _ = model_utils.make_and_restore_model(arch=models.mobilenet_v2(), dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/mobilenet_l2_eps3.ckpt',
                                parallel=False, add_custom_forward=True)
model = mobilenet.model
evaluate_accuracy(model, bs=4)

## Load ResNet Models

# ResNet-18

In [ ]:
## L-2 epsilon = 0

In [ ]:
imagenet_ds = ImageNet('/tmp/')
resnet , _ = model_utils.make_and_restore_model(arch="resnet18", dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet18_l2_eps0.ckpt',
                                parallel=False, add_custom_forward=True)
model = resnet.model

In [ ]:
evaluate_accuracy(model, bs=4)

In [ ]:
## L-2 epsilon = 3

In [ ]:
imagenet_ds = ImageNet('/tmp/')
resnet , _ = model_utils.make_and_restore_model(arch="resnet18", dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet-18-l2-eps3.ckpt',
                                parallel=False, add_custom_forward=True)
model = resnet.model

evaluate_accuracy(model, bs=4)

In [ ]:
# Linf epsilon = 0.5/255

In [ ]:
imagenet_ds = ImageNet('/tmp/')
resnet , _ = model_utils.make_and_restore_model(arch="resnet18", dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet18_linf_eps0.5.ckpt',
                                parallel=False, add_custom_forward=True)
model = resnet.model

evaluate_accuracy(model, bs=4)

In [ ]:
# Linf epsilon = 8/255

In [ ]:
imagenet_ds = ImageNet('/tmp/')
resnet , _ = model_utils.make_and_restore_model(arch="resnet18", dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet18_linf_eps8.0.ckpt',
                                parallel=False, add_custom_forward=True)
model = resnet.model

evaluate_accuracy(model, bs=4)

# ResNet-50

In [ ]:
# L2 epsilon = 0

In [ ]:
imagenet_ds = ImageNet('/tmp/')
resnet , _ = model_utils.make_and_restore_model(arch="resnet50", dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet50_l2_eps0.ckpt',
                                parallel=False, add_custom_forward=True)
#model = resnet.model

In [ ]:



transform = transforms.Compose([
transforms.Resize((224, 224)),
transforms.ToTensor()])

model = resnet
evaluate_accuracy(model, bs=1, tf=transform)

In [ ]:
# L2 epsilon = 3

In [5]:
imagenet_ds = ImageNet('/tmp/')
resnet , _ = model_utils.make_and_restore_model(arch="resnet50", dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet50_l2_eps3.ckpt',
                                parallel=False, add_custom_forward=True)

transform = transforms.Compose([
transforms.Resize((224, 224)),
transforms.ToTensor()])

model = resnet
evaluate_accuracy(model, bs=4, tf=transform)

=> loading checkpoint '/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet50_l2_eps3.ckpt'
=> loaded checkpoint '/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet50_l2_eps3.ckpt' (epoch 90)
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
216
220
224
228
232
236
240
244
248
252
256
260
264
268
272
276
280
284
288
292
296
300
304
308
312
316
320
324
328
332
336
340
344
348
352
356
360
364
368
372
376
380
384
388
392
396
400
404
408
412
416
420
424
428
432
436
440
444
448
452
456
460
464
468
472
476
480
484
488
492
496
500
504
508
512
516
520
524
528
532
536
540
544
548
552
556
560
564
568
572
576
580
584
588
592
596
600
604
608
612
616
620
624
628
632
636
640
644
648
652
656
660
664
668
672
676
680
684
688
692
696
700
704
708
712
716
720
724
728
732
736
740
744
748
752
756
760
764
768
772
776
780
784
788
792
796
800
804
808
812
816
820
824
828
83

6636
6640
6644
6648
6652
6656
6660
6664
6668
6672
6676
6680
6684
6688
6692
6696
6700
6704
6708
6712
6716
6720
6724
6728
6732
6736
6740
6744
6748
6752
6756
6760
6764
6768
6772
6776
6780
6784
6788
6792
6796
6800
6804
6808
6812
6816
6820
6824
6828
6832
6836
6840
6844
6848
6852
6856
6860
6864
6868
6872
6876
6880
6884
6888
6892
6896
6900
6904
6908
6912
6916
6920
6924
6928
6932
6936
6940
6944
6948
6952
6956
6960
6964
6968
6972
6976
6980
6984
6988
6992
6996
7000
7004
7008
7012
7016
7020
7024
7028
7032
7036
7040
7044
7048
7052
7056
7060
7064
7068
7072
7076
7080
7084
7088
7092
7096
7100
7104
7108
7112
7116
7120
7124
7128
7132
7136
7140
7144
7148
7152
7156
7160
7164
7168
7172
7176
7180
7184
7188
7192
7196
7200
7204
7208
7212
7216
7220
7224
7228
7232
7236
7240
7244
7248
7252
7256
7260
7264
7268
7272
7276
7280
7284
7288
7292
7296
7300
7304
7308
7312
7316
7320
7324
7328
7332
7336
7340
7344
7348
7352
7356
7360
7364
7368
7372
7376
7380
7384
7388
7392
7396
7400
7404
7408
7412
7416
7420
7424
7428
7432


12664
12668
12672
12676
12680
12684
12688
12692
12696
12700
12704
12708
12712
12716
12720
12724
12728
12732
12736
12740
12744
12748
12752
12756
12760
12764
12768
12772
12776
12780
12784
12788
12792
12796
12800
12804
12808
12812
12816
12820
12824
12828
12832
12836
12840
12844
12848
12852
12856
12860
12864
12868
12872
12876
12880
12884
12888
12892
12896
12900
12904
12908
12912
12916
12920
12924
12928
12932
12936
12940
12944
12948
12952
12956
12960
12964
12968
12972
12976
12980
12984
12988
12992
12996
13000
13004
13008
13012
13016
13020
13024
13028
13032
13036
13040
13044
13048
13052
13056
13060
13064
13068
13072
13076
13080
13084
13088
13092
13096
13100
13104
13108
13112
13116
13120
13124
13128
13132
13136
13140
13144
13148
13152
13156
13160
13164
13168
13172
13176
13180
13184
13188
13192
13196
13200
13204
13208
13212
13216
13220
13224
13228
13232
13236
13240
13244
13248
13252
13256
13260
13264
13268
13272
13276
13280
13284
13288
13292
13296
13300
13304
13308
13312
13316
13320
13324
1332

18140
18144
18148
18152
18156
18160
18164
18168
18172
18176
18180
18184
18188
18192
18196
18200
18204
18208
18212
18216
18220
18224
18228
18232
18236
18240
18244
18248
18252
18256
18260
18264
18268
18272
18276
18280
18284
18288
18292
18296
18300
18304
18308
18312
18316
18320
18324
18328
18332
18336
18340
18344
18348
18352
18356
18360
18364
18368
18372
18376
18380
18384
18388
18392
18396
18400
18404
18408
18412
18416
18420
18424
18428
18432
18436
18440
18444
18448
18452
18456
18460
18464
18468
18472
18476
18480
18484
18488
18492
18496
18500
18504
18508
18512
18516
18520
18524
18528
18532
18536
18540
18544
18548
18552
18556
18560
18564
18568
18572
18576
18580
18584
18588
18592
18596
18600
18604
18608
18612
18616
18620
18624
18628
18632
18636
18640
18644
18648
18652
18656
18660
18664
18668
18672
18676
18680
18684
18688
18692
18696
18700
18704
18708
18712
18716
18720
18724
18728
18732
18736
18740
18744
18748
18752
18756
18760
18764
18768
18772
18776
18780
18784
18788
18792
18796
18800
1880

23612
23616
23620
23624
23628
23632
23636
23640
23644
23648
23652
23656
23660
23664
23668
23672
23676
23680
23684
23688
23692
23696
23700
23704
23708
23712
23716
23720
23724
23728
23732
23736
23740
23744
23748
23752
23756
23760
23764
23768
23772
23776
23780
23784
23788
23792
23796
23800
23804
23808
23812
23816
23820
23824
23828
23832
23836
23840
23844
23848
23852
23856
23860
23864
23868
23872
23876
23880
23884
23888
23892
23896
23900
23904
23908
23912
23916
23920
23924
23928
23932
23936
23940
23944
23948
23952
23956
23960
23964
23968
23972
23976
23980
23984
23988
23992
23996
24000
24004
24008
24012
24016
24020
24024
24028
24032
24036
24040
24044
24048
24052
24056
24060
24064
24068
24072
24076
24080
24084
24088
24092
24096
24100
24104
24108
24112
24116
24120
24124
24128
24132
24136
24140
24144
24148
24152
24156
24160
24164
24168
24172
24176
24180
24184
24188
24192
24196
24200
24204
24208
24212
24216
24220
24224
24228
24232
24236
24240
24244
24248
24252
24256
24260
24264
24268
24272
2427

29088
29092
29096
29100
29104
29108
29112
29116
29120
29124
29128
29132
29136
29140
29144
29148
29152
29156
29160
29164
29168
29172
29176
29180
29184
29188
29192
29196
29200
29204
29208
29212
29216
29220
29224
29228
29232
29236
29240
29244
29248
29252
29256
29260
29264
29268
29272
29276
29280
29284
29288
29292
29296
29300
29304
29308
29312
29316
29320
29324
29328
29332
29336
29340
29344
29348
29352
29356
29360
29364
29368
29372
29376
29380
29384
29388
29392
29396
29400
29404
29408
29412
29416
29420
29424
29428
29432
29436
29440
29444
29448
29452
29456
29460
29464
29468
29472
29476
29480
29484
29488
29492
29496
29500
29504
29508
29512
29516
29520
29524
29528
29532
29536
29540
29544
29548
29552
29556
29560
29564
29568
29572
29576
29580
29584
29588
29592
29596
29600
29604
29608
29612
29616
29620
29624
29628
29632
29636
29640
29644
29648
29652
29656
29660
29664
29668
29672
29676
29680
29684
29688
29692
29696
29700
29704
29708
29712
29716
29720
29724
29728
29732
29736
29740
29744
29748
2975

34556
34560
34564
34568
34572
34576
34580
34584
34588
34592
34596
34600
34604
34608
34612
34616
34620
34624
34628
34632
34636
34640
34644
34648
34652
34656
34660
34664
34668
34672
34676
34680
34684
34688
34692
34696
34700
34704
34708
34712
34716
34720
34724
34728
34732
34736
34740
34744
34748
34752
34756
34760
34764
34768
34772
34776
34780
34784
34788
34792
34796
34800
34804
34808
34812
34816
34820
34824
34828
34832
34836
34840
34844
34848
34852
34856
34860
34864
34868
34872
34876
34880
34884
34888
34892
34896
34900
34904
34908
34912
34916
34920
34924
34928
34932
34936
34940
34944
34948
34952
34956
34960
34964
34968
34972
34976
34980
34984
34988
34992
34996
35000
35004
35008
35012
35016
35020
35024
35028
35032
35036
35040
35044
35048
35052
35056
35060
35064
35068
35072
35076
35080
35084
35088
35092
35096
35100
35104
35108
35112
35116
35120
35124
35128
35132
35136
35140
35144
35148
35152
35156
35160
35164
35168
35172
35176
35180
35184
35188
35192
35196
35200
35204
35208
35212
35216
3522

40028
40032
40036
40040
40044
40048
40052
40056
40060
40064
40068
40072
40076
40080
40084
40088
40092
40096
40100
40104
40108
40112
40116
40120
40124
40128
40132
40136
40140
40144
40148
40152
40156
40160
40164
40168
40172
40176
40180
40184
40188
40192
40196
40200
40204
40208
40212
40216
40220
40224
40228
40232
40236
40240
40244
40248
40252
40256
40260
40264
40268
40272
40276
40280
40284
40288
40292
40296
40300
40304
40308
40312
40316
40320
40324
40328
40332
40336
40340
40344
40348
40352
40356
40360
40364
40368
40372
40376
40380
40384
40388
40392
40396
40400
40404
40408
40412
40416
40420
40424
40428
40432
40436
40440
40444
40448
40452
40456
40460
40464
40468
40472
40476
40480
40484
40488
40492
40496
40500
40504
40508
40512
40516
40520
40524
40528
40532
40536
40540
40544
40548
40552
40556
40560
40564
40568
40572
40576
40580
40584
40588
40592
40596
40600
40604
40608
40612
40616
40620
40624
40628
40632
40636
40640
40644
40648
40652
40656
40660
40664
40668
40672
40676
40680
40684
40688
4069

45492
45496
45500
45504
45508
45512
45516
45520
45524
45528
45532
45536
45540
45544
45548
45552
45556
45560
45564
45568
45572
45576
45580
45584
45588
45592
45596
45600
45604
45608
45612
45616
45620
45624
45628
45632
45636
45640
45644
45648
45652
45656
45660
45664
45668
45672
45676
45680
45684
45688
45692
45696
45700
45704
45708
45712
45716
45720
45724
45728
45732
45736
45740
45744
45748
45752
45756
45760
45764
45768
45772
45776
45780
45784
45788
45792
45796
45800
45804
45808
45812
45816
45820
45824
45828
45832
45836
45840
45844
45848
45852
45856
45860
45864
45868
45872
45876
45880
45884
45888
45892
45896
45900
45904
45908
45912
45916
45920
45924
45928
45932
45936
45940
45944
45948
45952
45956
45960
45964
45968
45972
45976
45980
45984
45988
45992
45996
46000
46004
46008
46012
46016
46020
46024
46028
46032
46036
46040
46044
46048
46052
46056
46060
46064
46068
46072
46076
46080
46084
46088
46092
46096
46100
46104
46108
46112
46116
46120
46124
46128
46132
46136
46140
46144
46148
46152
4615

In [ ]:
# Linf epsilon = 0.5/255

In [6]:
imagenet_ds = ImageNet('/tmp/')
resnet , _ = model_utils.make_and_restore_model(arch="resnet50", dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet50_linf_eps0.5.ckpt',
                                parallel=False, add_custom_forward=True)
model = resnet
transform = transforms.Compose([
transforms.Resize((224, 224)),
transforms.ToTensor()])

evaluate_accuracy(model, bs=4, tf=transform)

=> loading checkpoint '/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet50_linf_eps0.5.ckpt'
=> loaded checkpoint '/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet50_linf_eps0.5.ckpt' (epoch 90)
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
216
220
224
228
232
236
240
244
248
252
256
260
264
268
272
276
280
284
288
292
296
300
304
308
312
316
320
324
328
332
336
340
344
348
352
356
360
364
368
372
376
380
384
388
392
396
400
404
408
412
416
420
424
428
432
436
440
444
448
452
456
460
464
468
472
476
480
484
488
492
496
500
504
508
512
516
520
524
528
532
536
540
544
548
552
556
560
564
568
572
576
580
584
588
592
596
600
604
608
612
616
620
624
628
632
636
640
644
648
652
656
660
664
668
672
676
680
684
688
692
696
700
704
708
712
716
720
724
728
732
736
740
744
748
752
756
760
764
768
772
776
780
784
788
792
796
800
804
808
812
816
820
82

6624
6628
6632
6636
6640
6644
6648
6652
6656
6660
6664
6668
6672
6676
6680
6684
6688
6692
6696
6700
6704
6708
6712
6716
6720
6724
6728
6732
6736
6740
6744
6748
6752
6756
6760
6764
6768
6772
6776
6780
6784
6788
6792
6796
6800
6804
6808
6812
6816
6820
6824
6828
6832
6836
6840
6844
6848
6852
6856
6860
6864
6868
6872
6876
6880
6884
6888
6892
6896
6900
6904
6908
6912
6916
6920
6924
6928
6932
6936
6940
6944
6948
6952
6956
6960
6964
6968
6972
6976
6980
6984
6988
6992
6996
7000
7004
7008
7012
7016
7020
7024
7028
7032
7036
7040
7044
7048
7052
7056
7060
7064
7068
7072
7076
7080
7084
7088
7092
7096
7100
7104
7108
7112
7116
7120
7124
7128
7132
7136
7140
7144
7148
7152
7156
7160
7164
7168
7172
7176
7180
7184
7188
7192
7196
7200
7204
7208
7212
7216
7220
7224
7228
7232
7236
7240
7244
7248
7252
7256
7260
7264
7268
7272
7276
7280
7284
7288
7292
7296
7300
7304
7308
7312
7316
7320
7324
7328
7332
7336
7340
7344
7348
7352
7356
7360
7364
7368
7372
7376
7380
7384
7388
7392
7396
7400
7404
7408
7412
7416
7420


12660
12664
12668
12672
12676
12680
12684
12688
12692
12696
12700
12704
12708
12712
12716
12720
12724
12728
12732
12736
12740
12744
12748
12752
12756
12760
12764
12768
12772
12776
12780
12784
12788
12792
12796
12800
12804
12808
12812
12816
12820
12824
12828
12832
12836
12840
12844
12848
12852
12856
12860
12864
12868
12872
12876
12880
12884
12888
12892
12896
12900
12904
12908
12912
12916
12920
12924
12928
12932
12936
12940
12944
12948
12952
12956
12960
12964
12968
12972
12976
12980
12984
12988
12992
12996
13000
13004
13008
13012
13016
13020
13024
13028
13032
13036
13040
13044
13048
13052
13056
13060
13064
13068
13072
13076
13080
13084
13088
13092
13096
13100
13104
13108
13112
13116
13120
13124
13128
13132
13136
13140
13144
13148
13152
13156
13160
13164
13168
13172
13176
13180
13184
13188
13192
13196
13200
13204
13208
13212
13216
13220
13224
13228
13232
13236
13240
13244
13248
13252
13256
13260
13264
13268
13272
13276
13280
13284
13288
13292
13296
13300
13304
13308
13312
13316
13320
1332

18140
18144
18148
18152
18156
18160
18164
18168
18172
18176
18180
18184
18188
18192
18196
18200
18204
18208
18212
18216
18220
18224
18228
18232
18236
18240
18244
18248
18252
18256
18260
18264
18268
18272
18276
18280
18284
18288
18292
18296
18300
18304
18308
18312
18316
18320
18324
18328
18332
18336
18340
18344
18348
18352
18356
18360
18364
18368
18372
18376
18380
18384
18388
18392
18396
18400
18404
18408
18412
18416
18420
18424
18428
18432
18436
18440
18444
18448
18452
18456
18460
18464
18468
18472
18476
18480
18484
18488
18492
18496
18500
18504
18508
18512
18516
18520
18524
18528
18532
18536
18540
18544
18548
18552
18556
18560
18564
18568
18572
18576
18580
18584
18588
18592
18596
18600
18604
18608
18612
18616
18620
18624
18628
18632
18636
18640
18644
18648
18652
18656
18660
18664
18668
18672
18676
18680
18684
18688
18692
18696
18700
18704
18708
18712
18716
18720
18724
18728
18732
18736
18740
18744
18748
18752
18756
18760
18764
18768
18772
18776
18780
18784
18788
18792
18796
18800
1880

23608
23612
23616
23620
23624
23628
23632
23636
23640
23644
23648
23652
23656
23660
23664
23668
23672
23676
23680
23684
23688
23692
23696
23700
23704
23708
23712
23716
23720
23724
23728
23732
23736
23740
23744
23748
23752
23756
23760
23764
23768
23772
23776
23780
23784
23788
23792
23796
23800
23804
23808
23812
23816
23820
23824
23828
23832
23836
23840
23844
23848
23852
23856
23860
23864
23868
23872
23876
23880
23884
23888
23892
23896
23900
23904
23908
23912
23916
23920
23924
23928
23932
23936
23940
23944
23948
23952
23956
23960
23964
23968
23972
23976
23980
23984
23988
23992
23996
24000
24004
24008
24012
24016
24020
24024
24028
24032
24036
24040
24044
24048
24052
24056
24060
24064
24068
24072
24076
24080
24084
24088
24092
24096
24100
24104
24108
24112
24116
24120
24124
24128
24132
24136
24140
24144
24148
24152
24156
24160
24164
24168
24172
24176
24180
24184
24188
24192
24196
24200
24204
24208
24212
24216
24220
24224
24228
24232
24236
24240
24244
24248
24252
24256
24260
24264
24268
2427

29072
29076
29080
29084
29088
29092
29096
29100
29104
29108
29112
29116
29120
29124
29128
29132
29136
29140
29144
29148
29152
29156
29160
29164
29168
29172
29176
29180
29184
29188
29192
29196
29200
29204
29208
29212
29216
29220
29224
29228
29232
29236
29240
29244
29248
29252
29256
29260
29264
29268
29272
29276
29280
29284
29288
29292
29296
29300
29304
29308
29312
29316
29320
29324
29328
29332
29336
29340
29344
29348
29352
29356
29360
29364
29368
29372
29376
29380
29384
29388
29392
29396
29400
29404
29408
29412
29416
29420
29424
29428
29432
29436
29440
29444
29448
29452
29456
29460
29464
29468
29472
29476
29480
29484
29488
29492
29496
29500
29504
29508
29512
29516
29520
29524
29528
29532
29536
29540
29544
29548
29552
29556
29560
29564
29568
29572
29576
29580
29584
29588
29592
29596
29600
29604
29608
29612
29616
29620
29624
29628
29632
29636
29640
29644
29648
29652
29656
29660
29664
29668
29672
29676
29680
29684
29688
29692
29696
29700
29704
29708
29712
29716
29720
29724
29728
29732
2973

34536
34540
34544
34548
34552
34556
34560
34564
34568
34572
34576
34580
34584
34588
34592
34596
34600
34604
34608
34612
34616
34620
34624
34628
34632
34636
34640
34644
34648
34652
34656
34660
34664
34668
34672
34676
34680
34684
34688
34692
34696
34700
34704
34708
34712
34716
34720
34724
34728
34732
34736
34740
34744
34748
34752
34756
34760
34764
34768
34772
34776
34780
34784
34788
34792
34796
34800
34804
34808
34812
34816
34820
34824
34828
34832
34836
34840
34844
34848
34852
34856
34860
34864
34868
34872
34876
34880
34884
34888
34892
34896
34900
34904
34908
34912
34916
34920
34924
34928
34932
34936
34940
34944
34948
34952
34956
34960
34964
34968
34972
34976
34980
34984
34988
34992
34996
35000
35004
35008
35012
35016
35020
35024
35028
35032
35036
35040
35044
35048
35052
35056
35060
35064
35068
35072
35076
35080
35084
35088
35092
35096
35100
35104
35108
35112
35116
35120
35124
35128
35132
35136
35140
35144
35148
35152
35156
35160
35164
35168
35172
35176
35180
35184
35188
35192
35196
3520

40004
40008
40012
40016
40020
40024
40028
40032
40036
40040
40044
40048
40052
40056
40060
40064
40068
40072
40076
40080
40084
40088
40092
40096
40100
40104
40108
40112
40116
40120
40124
40128
40132
40136
40140
40144
40148
40152
40156
40160
40164
40168
40172
40176
40180
40184
40188
40192
40196
40200
40204
40208
40212
40216
40220
40224
40228
40232
40236
40240
40244
40248
40252
40256
40260
40264
40268
40272
40276
40280
40284
40288
40292
40296
40300
40304
40308
40312
40316
40320
40324
40328
40332
40336
40340
40344
40348
40352
40356
40360
40364
40368
40372
40376
40380
40384
40388
40392
40396
40400
40404
40408
40412
40416
40420
40424
40428
40432
40436
40440
40444
40448
40452
40456
40460
40464
40468
40472
40476
40480
40484
40488
40492
40496
40500
40504
40508
40512
40516
40520
40524
40528
40532
40536
40540
40544
40548
40552
40556
40560
40564
40568
40572
40576
40580
40584
40588
40592
40596
40600
40604
40608
40612
40616
40620
40624
40628
40632
40636
40640
40644
40648
40652
40656
40660
40664
4066

45468
45472
45476
45480
45484
45488
45492
45496
45500
45504
45508
45512
45516
45520
45524
45528
45532
45536
45540
45544
45548
45552
45556
45560
45564
45568
45572
45576
45580
45584
45588
45592
45596
45600
45604
45608
45612
45616
45620
45624
45628
45632
45636
45640
45644
45648
45652
45656
45660
45664
45668
45672
45676
45680
45684
45688
45692
45696
45700
45704
45708
45712
45716
45720
45724
45728
45732
45736
45740
45744
45748
45752
45756
45760
45764
45768
45772
45776
45780
45784
45788
45792
45796
45800
45804
45808
45812
45816
45820
45824
45828
45832
45836
45840
45844
45848
45852
45856
45860
45864
45868
45872
45876
45880
45884
45888
45892
45896
45900
45904
45908
45912
45916
45920
45924
45928
45932
45936
45940
45944
45948
45952
45956
45960
45964
45968
45972
45976
45980
45984
45988
45992
45996
46000
46004
46008
46012
46016
46020
46024
46028
46032
46036
46040
46044
46048
46052
46056
46060
46064
46068
46072
46076
46080
46084
46088
46092
46096
46100
46104
46108
46112
46116
46120
46124
46128
4613

In [ ]:
# Linf epsilon = 8/255

In [7]:
imagenet_ds = ImageNet('/tmp/')
resnet , _ = model_utils.make_and_restore_model(arch="resnet50", dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet50_linf_eps8.0.ckpt',
                                parallel=False, add_custom_forward=True)
model = resnet
transform = transforms.Compose([
transforms.Resize((224, 224)),
transforms.ToTensor()])

evaluate_accuracy(model, bs=4, tf=transform)

=> loading checkpoint '/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet50_linf_eps8.0.ckpt'
=> loaded checkpoint '/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet50_linf_eps8.0.ckpt' (epoch 90)
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
216
220
224
228
232
236
240
244
248
252
256
260
264
268
272
276
280
284
288
292
296
300
304
308
312
316
320
324
328
332
336
340
344
348
352
356
360
364
368
372
376
380
384
388
392
396
400
404
408
412
416
420
424
428
432
436
440
444
448
452
456
460
464
468
472
476
480
484
488
492
496
500
504
508
512
516
520
524
528
532
536
540
544
548
552
556
560
564
568
572
576
580
584
588
592
596
600
604
608
612
616
620
624
628
632
636
640
644
648
652
656
660
664
668
672
676
680
684
688
692
696
700
704
708
712
716
720
724
728
732
736
740
744
748
752
756
760
764
768
772
776
780
784
788
792
796
800
804
808
812
816
820
82

6616
6620
6624
6628
6632
6636
6640
6644
6648
6652
6656
6660
6664
6668
6672
6676
6680
6684
6688
6692
6696
6700
6704
6708
6712
6716
6720
6724
6728
6732
6736
6740
6744
6748
6752
6756
6760
6764
6768
6772
6776
6780
6784
6788
6792
6796
6800
6804
6808
6812
6816
6820
6824
6828
6832
6836
6840
6844
6848
6852
6856
6860
6864
6868
6872
6876
6880
6884
6888
6892
6896
6900
6904
6908
6912
6916
6920
6924
6928
6932
6936
6940
6944
6948
6952
6956
6960
6964
6968
6972
6976
6980
6984
6988
6992
6996
7000
7004
7008
7012
7016
7020
7024
7028
7032
7036
7040
7044
7048
7052
7056
7060
7064
7068
7072
7076
7080
7084
7088
7092
7096
7100
7104
7108
7112
7116
7120
7124
7128
7132
7136
7140
7144
7148
7152
7156
7160
7164
7168
7172
7176
7180
7184
7188
7192
7196
7200
7204
7208
7212
7216
7220
7224
7228
7232
7236
7240
7244
7248
7252
7256
7260
7264
7268
7272
7276
7280
7284
7288
7292
7296
7300
7304
7308
7312
7316
7320
7324
7328
7332
7336
7340
7344
7348
7352
7356
7360
7364
7368
7372
7376
7380
7384
7388
7392
7396
7400
7404
7408
7412


12652
12656
12660
12664
12668
12672
12676
12680
12684
12688
12692
12696
12700
12704
12708
12712
12716
12720
12724
12728
12732
12736
12740
12744
12748
12752
12756
12760
12764
12768
12772
12776
12780
12784
12788
12792
12796
12800
12804
12808
12812
12816
12820
12824
12828
12832
12836
12840
12844
12848
12852
12856
12860
12864
12868
12872
12876
12880
12884
12888
12892
12896
12900
12904
12908
12912
12916
12920
12924
12928
12932
12936
12940
12944
12948
12952
12956
12960
12964
12968
12972
12976
12980
12984
12988
12992
12996
13000
13004
13008
13012
13016
13020
13024
13028
13032
13036
13040
13044
13048
13052
13056
13060
13064
13068
13072
13076
13080
13084
13088
13092
13096
13100
13104
13108
13112
13116
13120
13124
13128
13132
13136
13140
13144
13148
13152
13156
13160
13164
13168
13172
13176
13180
13184
13188
13192
13196
13200
13204
13208
13212
13216
13220
13224
13228
13232
13236
13240
13244
13248
13252
13256
13260
13264
13268
13272
13276
13280
13284
13288
13292
13296
13300
13304
13308
13312
1331

18116
18120
18124
18128
18132
18136
18140
18144
18148
18152
18156
18160
18164
18168
18172
18176
18180
18184
18188
18192
18196
18200
18204
18208
18212
18216
18220
18224
18228
18232
18236
18240
18244
18248
18252
18256
18260
18264
18268
18272
18276
18280
18284
18288
18292
18296
18300
18304
18308
18312
18316
18320
18324
18328
18332
18336
18340
18344
18348
18352
18356
18360
18364
18368
18372
18376
18380
18384
18388
18392
18396
18400
18404
18408
18412
18416
18420
18424
18428
18432
18436
18440
18444
18448
18452
18456
18460
18464
18468
18472
18476
18480
18484
18488
18492
18496
18500
18504
18508
18512
18516
18520
18524
18528
18532
18536
18540
18544
18548
18552
18556
18560
18564
18568
18572
18576
18580
18584
18588
18592
18596
18600
18604
18608
18612
18616
18620
18624
18628
18632
18636
18640
18644
18648
18652
18656
18660
18664
18668
18672
18676
18680
18684
18688
18692
18696
18700
18704
18708
18712
18716
18720
18724
18728
18732
18736
18740
18744
18748
18752
18756
18760
18764
18768
18772
18776
1878

23584
23588
23592
23596
23600
23604
23608
23612
23616
23620
23624
23628
23632
23636
23640
23644
23648
23652
23656
23660
23664
23668
23672
23676
23680
23684
23688
23692
23696
23700
23704
23708
23712
23716
23720
23724
23728
23732
23736
23740
23744
23748
23752
23756
23760
23764
23768
23772
23776
23780
23784
23788
23792
23796
23800
23804
23808
23812
23816
23820
23824
23828
23832
23836
23840
23844
23848
23852
23856
23860
23864
23868
23872
23876
23880
23884
23888
23892
23896
23900
23904
23908
23912
23916
23920
23924
23928
23932
23936
23940
23944
23948
23952
23956
23960
23964
23968
23972
23976
23980
23984
23988
23992
23996
24000
24004
24008
24012
24016
24020
24024
24028
24032
24036
24040
24044
24048
24052
24056
24060
24064
24068
24072
24076
24080
24084
24088
24092
24096
24100
24104
24108
24112
24116
24120
24124
24128
24132
24136
24140
24144
24148
24152
24156
24160
24164
24168
24172
24176
24180
24184
24188
24192
24196
24200
24204
24208
24212
24216
24220
24224
24228
24232
24236
24240
24244
2424

29052
29056
29060
29064
29068
29072
29076
29080
29084
29088
29092
29096
29100
29104
29108
29112
29116
29120
29124
29128
29132
29136
29140
29144
29148
29152
29156
29160
29164
29168
29172
29176
29180
29184
29188
29192
29196
29200
29204
29208
29212
29216
29220
29224
29228
29232
29236
29240
29244
29248
29252
29256
29260
29264
29268
29272
29276
29280
29284
29288
29292
29296
29300
29304
29308
29312
29316
29320
29324
29328
29332
29336
29340
29344
29348
29352
29356
29360
29364
29368
29372
29376
29380
29384
29388
29392
29396
29400
29404
29408
29412
29416
29420
29424
29428
29432
29436
29440
29444
29448
29452
29456
29460
29464
29468
29472
29476
29480
29484
29488
29492
29496
29500
29504
29508
29512
29516
29520
29524
29528
29532
29536
29540
29544
29548
29552
29556
29560
29564
29568
29572
29576
29580
29584
29588
29592
29596
29600
29604
29608
29612
29616
29620
29624
29628
29632
29636
29640
29644
29648
29652
29656
29660
29664
29668
29672
29676
29680
29684
29688
29692
29696
29700
29704
29708
29712
2971

34520
34524
34528
34532
34536
34540
34544
34548
34552
34556
34560
34564
34568
34572
34576
34580
34584
34588
34592
34596
34600
34604
34608
34612
34616
34620
34624
34628
34632
34636
34640
34644
34648
34652
34656
34660
34664
34668
34672
34676
34680
34684
34688
34692
34696
34700
34704
34708
34712
34716
34720
34724
34728
34732
34736
34740
34744
34748
34752
34756
34760
34764
34768
34772
34776
34780
34784
34788
34792
34796
34800
34804
34808
34812
34816
34820
34824
34828
34832
34836
34840
34844
34848
34852
34856
34860
34864
34868
34872
34876
34880
34884
34888
34892
34896
34900
34904
34908
34912
34916
34920
34924
34928
34932
34936
34940
34944
34948
34952
34956
34960
34964
34968
34972
34976
34980
34984
34988
34992
34996
35000
35004
35008
35012
35016
35020
35024
35028
35032
35036
35040
35044
35048
35052
35056
35060
35064
35068
35072
35076
35080
35084
35088
35092
35096
35100
35104
35108
35112
35116
35120
35124
35128
35132
35136
35140
35144
35148
35152
35156
35160
35164
35168
35172
35176
35180
3518

39984
39988
39992
39996
40000
40004
40008
40012
40016
40020
40024
40028
40032
40036
40040
40044
40048
40052
40056
40060
40064
40068
40072
40076
40080
40084
40088
40092
40096
40100
40104
40108
40112
40116
40120
40124
40128
40132
40136
40140
40144
40148
40152
40156
40160
40164
40168
40172
40176
40180
40184
40188
40192
40196
40200
40204
40208
40212
40216
40220
40224
40228
40232
40236
40240
40244
40248
40252
40256
40260
40264
40268
40272
40276
40280
40284
40288
40292
40296
40300
40304
40308
40312
40316
40320
40324
40328
40332
40336
40340
40344
40348
40352
40356
40360
40364
40368
40372
40376
40380
40384
40388
40392
40396
40400
40404
40408
40412
40416
40420
40424
40428
40432
40436
40440
40444
40448
40452
40456
40460
40464
40468
40472
40476
40480
40484
40488
40492
40496
40500
40504
40508
40512
40516
40520
40524
40528
40532
40536
40540
40544
40548
40552
40556
40560
40564
40568
40572
40576
40580
40584
40588
40592
40596
40600
40604
40608
40612
40616
40620
40624
40628
40632
40636
40640
40644
4064

45452
45456
45460
45464
45468
45472
45476
45480
45484
45488
45492
45496
45500
45504
45508
45512
45516
45520
45524
45528
45532
45536
45540
45544
45548
45552
45556
45560
45564
45568
45572
45576
45580
45584
45588
45592
45596
45600
45604
45608
45612
45616
45620
45624
45628
45632
45636
45640
45644
45648
45652
45656
45660
45664
45668
45672
45676
45680
45684
45688
45692
45696
45700
45704
45708
45712
45716
45720
45724
45728
45732
45736
45740
45744
45748
45752
45756
45760
45764
45768
45772
45776
45780
45784
45788
45792
45796
45800
45804
45808
45812
45816
45820
45824
45828
45832
45836
45840
45844
45848
45852
45856
45860
45864
45868
45872
45876
45880
45884
45888
45892
45896
45900
45904
45908
45912
45916
45920
45924
45928
45932
45936
45940
45944
45948
45952
45956
45960
45964
45968
45972
45976
45980
45984
45988
45992
45996
46000
46004
46008
46012
46016
46020
46024
46028
46032
46036
46040
46044
46048
46052
46056
46060
46064
46068
46072
46076
46080
46084
46088
46092
46096
46100
46104
46108
46112
4611